# 01-Nature Tutorial- Electronic structure

#### Uncomment the cell below to pip install the necessary modules if not already installed

#### Note: Works with Qiskit Version 1.4.1

In [1]:
# %pip install qiskit-nature
# %pip install qiskit-algorithms
# %pip install qiskit==1.4.1
# %pip install --prefer-binary pyscf

#### Restart the kernel after installing any of the missing packages

### Introduction

In this notebook we will be exploring the electronic structure of atoms and molecules. While we only show the $H_2$ molecule in this notebook, you are encouraged to think and explore with other molecules or single atoms. This will require some chemistry knowledge, but there are many resources available online.

For this notebook, we will need to cover some "basic" ideas regarding electronic structure of molecules which is largely governed by the energy of the molecule and the molecular orbitals. We can begin to describe these using the molecular Hamiltonian shown below.

$$
\mathcal{H} = - \sum_I \frac{\nabla_{R_I}^2}{M_I} - \sum_i \frac{\nabla_{r_i}^2}{m_e} - \sum_I\sum_i  \frac{Z_I e^2}{|R_I-r_i|} + \sum_i \sum_{j>i} \frac{e^2}{|r_i-r_j|} + \sum_I\sum_{J>I} \frac{Z_I Z_J e^2}{|R_I-R_J|}
$$

Yes, that is hard to look at. Let's break it down to something more manageable.

$$
\hat{H} = \hat{K}_N + \hat{K}_e + \hat{V}_{eN} + \hat{V}_{ee} + \hat{V}_{NN}
$$

These are directly transferable. Each term represents a different interaction within the molecule as defined below:

$\hat{K}_N$: Nuclear kinetic energy operator

$\hat{K}_e$: Electron kinetic energy operator

$\hat{V}_{eN}$: Potential energy between electrons and nuclei

$\hat{V}_{ee}$: Potential energy between electrons

$\hat{V}_{NN}$: Potential energy between nuclei

Now that we've made some sense of the above, let's reduce it down further. The nucleus of an atom has a velocity significantly lower than that of an electron. Relative to the electron, we could treat the nucleus as though it is in a fixed position. So we can drop the nuclear kinetic energy term from the Hamiltonian. Our last simplification will come from dropping the potential energy of the nuclei. We can do this because we are fixing our nuclei, causing this term to become a constant value that will only shift our eigenvalue amounts by some fixed amount. This simplification is part of the [Born-Oppenheimer Approximation](https://en.wikipedia.org/wiki/Born%E2%80%93Oppenheimer_approximation), and despite being an approximation, is surprisingly accurate. 

Our simplified equation then becomes 

$$
\hat{H} = \hat{K}_e + \hat{V}_{eN} + \hat{V}_{ee}
$$

Where
$$
\hat{K}_e = - \sum_i \frac{\nabla_{r_i}^2}{m_e}
$$
$$
\hat{V}_{eN} = - \sum_I\sum_i  \frac{Z_I e^2}{|R_I-r_i|}
$$
$$
\hat{V}_{ee} = \sum_i \sum_{j>i} \frac{e^2}{|r_i-r_j|}
$$

With our new Hamiltonian operator, we can create a time independent Schrödinger equation (TISE)

$$
\mathcal{H}_{el} \ket{\Psi_n} = E_n\ket{\Psi_n}
$$

where 

$$
\mathcal{H}_{el} = - \sum_i \frac{\nabla_{r_i}^2}{m_e} - \sum_I\sum_i  \frac{Z_I e^2}{|R_I-r_i|} + \sum_i \sum_{j>i} \frac{e^2}{|r_i-r_j|}
$$

We are interested in the ground state energy for the system given by:

$$
E_0 = \frac{\bra{\Psi_0}H_{el}\ket{\Psi_0}}{\langle \Psi_0 | \Psi_0 \rangle}
$$

Since the dimensionality of this problem grows exponentially with the number of degrees of freedom (think $2^N$ with N being the system size), we encounter a problem. We can approach this by preparing $\Psi_0$ on a quantum computer and measure the Hamiltonian expectation value ($E_0$) directly. Thus, our last step of this problem involves applying the Hartree-Fock solution.

### Hartree-Fock 

The [Hartree-Fock (HF)](https://en.wikipedia.org/wiki/Hartree%E2%80%93Fock_method) method is another approximation method for an N-body problem that works by breaking the full problem down into N one-body problems. This means each electron is treated as though it is evolving in the mean-field of the other electrons. Classically solving the HF equations leads to the exact exchange energy in an efficient manner, but does not include any electron correlations. Therefore, we can start with this and add the correlations later.

We can re-express the Hamiltonian in the basis of the solutions of the HF method, this is called the Molecular Orbitals (MOs):

$$
\hat{H}_{elec}=\sum_{pq} h_{pq} \hat{a}^{\dagger}_p \hat{a}_q + 
\frac{1}{2} \sum_{pqrs} h_{pqrs}  \hat{a}^{\dagger}_p \hat{a}^{\dagger}_q \hat{a}_r  \hat{a}_s
$$

with 1-body integrals 

$$
h_{pq} = \int \phi^*_p(r) \left( -\frac{1}{2} \nabla^2 - \sum_{I} \frac{Z_I}{R_I- r} \right)   \phi_q(r)dr
$$

and 2-body integrals

$$
h_{pqrs} = \int \frac{\phi^*_p(r_1)  \phi^*_q(r_2) \phi_r(r_2)  \phi_s(r_1)}{|r_1-r_2|}dr_1dr_2.
$$

The MOs ($\phi_u$) can be occupied or unoccupied (virtual). One MO can contain 2 electrons. With what we will be doing, we actually care more about the Spin Orbitals which are associated with a spin up ($\alpha$) or spin down ($\beta$) electron. Thus Spin Orbitals can contain one electron or be unoccupied.

> **Note**: when referring to the number of orbitals, we will be using the number of spatial orbitals. These orbitals exist in a Cartesian space where molecular or spin do not matter. Each spatial orbital is then split into two Spin Orbitals.

Now that we have an introductory understanding of the electronic structure at the quantum level, we can begin to concretely bring this theory into reality with Qiskit.

In [2]:
# This code is from:
# https://qiskit-community.github.io/qiskit-nature/tutorials/01_electronic_structure.html

#### Obtain an initial Hartree-Fock solution

There are several classical codes that are capabable of finding the HF solutions. For Qiskit we have:
<ul>
    <li>Gaussian</li>
    <li>Psi4</li>
    <li>PySCF</li>
</ul>

We will set up a PySCF driver for the hydrogen molecule at equilibrium bond length of 0.735 angstrom (Å) in the singlet state and with no charge.

In [3]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver

driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

### ElectronicStructureProblem

With our driver created, we now have a variable that holds our `ElectronicStructureProblem`. Most of this notebook will be spent exploring the different features and attributes of this problem instance. First we will run the driver and print out the problem as the object type and memory location.

In [4]:
problem = driver.run()
print(problem)

Our most important aspect of this problem is the internal Hamiltonian. This is defined as the `ElectronicEnergy` Hamiltonian. The `ElectronicStructureProblem` class is able to generate the second-quantized operator from the 1- and 2-body integrals which the classical code has computed for us.

>**Important**: The container class for the integral coefficients (`PolynomialTensor`) requires the 2-body terms to be provided in physicist order! This is seen in the way the alpha coefficients print out.

Physicist order follows the notation in the integral above [p, q, r, s] So for the one-electron integral the coefficients will print out as [p, q] and for the two-electron integral it will print out as [p, q, r, s]. This is an important distinction from chemistry order which changes the ordering of the indices.

In [5]:
hamiltonian = problem.hamiltonian

coefficients = hamiltonian.electronic_integrals
print(coefficients.alpha)

Polynomial Tensor
 "+-":
array([[-1.25633907e+00, -1.37083854e-17],
       [-6.07732712e-17, -4.71896007e-01]])
 "++--":
array([6.75710155e-01, 1.38777878e-16, 1.80931200e-01, 6.64581730e-01,
       5.55111512e-17, 6.98573723e-01])


In [6]:
# The second quantized operator of the Hamiltonian. Does not include the nuclear repulsion energy

second_q_op = hamiltonian.second_q_op()
print(second_q_op)

Fermionic Operator
number spin orbitals=4, number terms=36
  -1.25633907300325 * ( +_0 -_0 )
+ -0.47189600728114184 * ( +_1 -_1 )
+ -1.25633907300325 * ( +_2 -_2 )
+ -0.47189600728114184 * ( +_3 -_3 )
+ 0.33785507740175813 * ( +_0 +_0 -_0 -_0 )
+ 0.3322908651276482 * ( +_0 +_1 -_1 -_0 )
+ 0.33785507740175813 * ( +_0 +_2 -_2 -_0 )
+ 0.3322908651276482 * ( +_0 +_3 -_3 -_0 )
+ 0.09046559989211572 * ( +_0 +_0 -_1 -_1 )
+ 0.09046559989211572 * ( +_0 +_1 -_0 -_1 )
+ 0.09046559989211572 * ( +_0 +_2 -_3 -_1 )
+ 0.09046559989211572 * ( +_0 +_3 -_2 -_1 )
+ 0.09046559989211572 * ( +_1 +_0 -_1 -_0 )
+ 0.09046559989211572 * ( +_1 +_1 -_0 -_0 )
+ 0.09046559989211572 * ( +_1 +_2 -_3 -_0 )
+ 0.09046559989211572 * ( +_1 +_3 -_2 -_0 )
+ 0.3322908651276482 * ( +_1 +_0 -_0 -_1 )
+ 0.34928686136600884 * ( +_1 +_1 -_1 -_1 )
+ 0.3322908651276482 * ( +_1 +_2 -_2 -_1 )
+ 0.34928686136600884 * ( +_1 +_3 -_3 -_1 )
+ 0.33785507740175813 * ( +_2 +_0 -_0 -_2 )
+ 0.3322908651276482 * ( +_2 +_1 -_1 -_2 )
+ 0.33785507

In [7]:
# You can print out the nuclear repulsion energy directly

hamiltonian.nuclear_repulsion_energy  # NOT included in the second_q_op above

np.float64(0.7199689944489797)

In [8]:
# Prints out the information regarding the molecule initially created

problem.molecule

MoleculeInfo(symbols=['H', 'H'], coords=[(0.0, 0.0, 0.0), (0.0, 0.0, 1.3889487015553204)], multiplicity=1, charge=0, units=<DistanceUnit.BOHR: 'Bohr'>, masses=[np.int64(1), np.int64(1)])

In [9]:
# Prints out the reference energy for the ground state of the molecule.

problem.reference_energy

np.float64(-1.116998996754004)

In [10]:
# Number of particles in alpha- and beta-spin

problem.num_particles

(1, 1)

In [11]:
# Number of spatial orbitals in the system

problem.num_spatial_orbitals

2

In [12]:
# The electronic basis of all contained orbital coefficients

problem.basis

<ElectronicBasis.MO: 'molecular'>

The `ElectronicStructureProblem` also contains additional operator factories which will generate observables to be evaluated at the ground- and excited-states at the end of your computation. These are stored in the `properties` container as shown below.

In [13]:
# Container for additional observable operator factories

problem.properties

In [14]:
# The ParticleNumber property. Contains information about spatial orbitals

problem.properties.particle_number

In [15]:
# The AngularMomentum property

problem.properties.angular_momentum

In [16]:
# The Magnetization property

problem.properties.magnetization

In [17]:
# The ElectronicDipoleMoment property. 
# Contains information about the cartesian dipoles for the system

problem.properties.electronic_dipole_moment

### Solve the Problem

Now that we are familiar with some of the components of the `ElectronicStructureProblem` we can compute the ground-state of our problem instance. First we will create an Eigensolver using a `GroundStateEigenSolver` class that takes in a mapper and eigensolver function. In short, the [Jordan-Wigner mapper](https://en.wikipedia.org/wiki/Jordan%E2%80%93Wigner_transformation) will map spin operators onto fermionic (electron) creation and annihilation operators. The Eigensolver is a standard method using numpy built into the `qiskit_algorithms` module.

In [18]:
from qiskit_algorithms import NumPyMinimumEigensolver
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.second_q.mappers import JordanWignerMapper

solver = GroundStateEigensolver(
    JordanWignerMapper(),
    NumPyMinimumEigensolver(),
)

In [19]:
# Solve the problem!

result = solver.solve(problem)
print(result)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.857275030202
  - computed part:      -1.857275030202
~ Nuclear repulsion energy (Hartree): 0.719968994449
> Total ground state energy (Hartree): -1.137306035753
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  1.3889487]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  1.388948701555]
    - computed part:      [0.0  0.0  1.388948701555]
  > Dipole moment (a.u.): [0.0  0.0  -0.000000001555]  Total: 0.000000001555
                 (debye): [0.0  0.0  -0.000000003953]  Total: 0.000000003953
 


### Conclusion

Congratulations! You've completed an introductory exploration of the `ElectronicStructureProblem` for a Hydrogen Molecule! The final results above print out the ground state energy for the molecule based on the Hartree-Fock solution. The measured observables are the number of particles, total spin, spin-squared, and spin projection quantum number. Since this is a singlet state the value of zero is expected. The dipole moments give us information about the nuclei and electron dipoles. The units of a.u. are atomic units and debye is a non-SI unit for dipole moment. 1 a.u. is approximately 2.5417 Debye. These values are roughly 0 as the Hydrogen molecule is symmetric so no net charge separation is present.